# RNN
- 시계열 데이터를 처리하기에 좋은 뉴럴 네트워크 구조 (Recarrent Neural Network)
- Example
    - 번역기
    - 음악 생성기
    - 음성 인식
    - DNA 염기서열 분석

## RNN 특징
- CNN이 이미지 구역별로 같은 weight를 공유한다면,
- RNN은 시간 별로 같은 weight을 공유한다.
- 즉, 과거와 현재는 같은 weight를 공유한다.

## First Order System
- 현재 시간의 상태가 이전 시간의 상태와 관련이 있다고 가정
    - First-Order System
        $ x_t = f(x_(t-1)) $
    - 그림으로 표현하면
    - 이 시스템은 `**외부 입력**` 없이 자기 혼자서 돌아간다.
    - autonomous system
- 현재 시간의 상태가 이전 시간의 상태와, 현재의 입력에 관계가 있는 경우
    - 상태: x, 입력: u
    - 식으로 표현하면
        $ x_t = f(x_(t-1), u_t) $


## State-Space Model
- 어떤 시스템을 해석하기 위한 3요소:
    - $ x_t = f(x_(t-1), u_t) $
    - $ y_t = h(x_t) $









In [1]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=0, keepdims=True))
    return e_x / np.sum(e_x, axis=0, keepdims=True)

In [2]:
def rnn_cell_forward(xt, a_prev, parameters):
   Wax = parameters["Wax"]
   Waa = parameters["Waa"]
   Wya = parameters["Wya"]
   ba = parameters["ba"]
   by = parameters["by"]

   a_next = np.tanh(Wax @ xt + Waa @ a_prev + ba)
   yt_pred = softmax(Wya @ a_next + by)

   cache = (a_next, a_prev, xt, parameters)
   return a_next, yt_pred, cache

In [3]:
def rnn_forward(x, a0, parameters):

    caches = []
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape

    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))

    a_next = a0
    for t in range(T_x):
        xt = x[:, :, t]
        a_next, yt_pred, cache = rnn_cell_forward(xt, a_next, parameters)
        a[:, :, t] = a_next
        y_pred[:, :, t] = yt_pred
        caches.append(cache)

    caches = (caches, x)
    return a, y_pred, caches